**next level of SQL (Spark SQL) + Python Function based programming (Framework of Spark DSL) + Datawarehouse (Datalake+Lakehouse) -> Transformation & Analytics**

### Data munging is the process of converting raw data into a usable format by cleaning, transforming, and enriching it. ![](/Workspace/Users/sachinchitte4@gmail.com/Pratice/stage1.png)


**Passive Data Munging : Performing an (Data Exploration) exploratory data analysis of the raw data to identify the attributes and patterns.**
1. Visibily/Manually opening the file we found couple of data patterns (Manual Exploratory Data Analysis)
- It is a Structured data with comma seperator (CSV)
- No Header, No comments, footer is there in the data
- Total columns are (seperator + 1)
- Data Quality
- Null columns are there
- duplicate rows
- format issues are there (age is not in number format eg. 7-7)
- Uniformity issues (Artist, artist)
- Number of columns are more or less than the expected
- eg. 4000011,Francis,McNamara,47,Therapist,NewYork & 4000014,Beth,Woodard,65
- Identification of data type

**2. Programatically lets try to find couple of data patterns applying EDA - passively (without modifying, just for description)**

In [0]:
%sql
create catalog if not exists BreadandButter; 
create database if not exists BreadandButter.Data_Ingestion_DB; 
create volume if not exists BreadandButter.Data_Ingestion_DB.Data_DE_VL;

2. Programatically lets try to find couple of data patterns applying EDA - passively (without modifying, just for description).

In [0]:
rawdf1=spark.read.csv("/Volumes/breadandbutter/data_ingestion_db/data_de_vl/custsmodified_NY",header=False,inferSchema=True).toDF("id","firstname","lastname","age","profession")
#rawdf1.show(20,False)
display(rawdf1.take(20))
display(rawdf1.sample(.1))
#inferSchema=True ->Spark scans data and assigns data types automatically
#sample(.1) #Randomly selects ~10% of rows -> Data inspection
#take(20) -> Displaying first 20 rows (Action)

In [0]:
#Important passive EDA structure functions we can use
rawdf1.printSchema() #I am realizing the id & age columns are having some non numerical values (supposed to be numeric)

#Column names in exact order
print(rawdf1.columns) #I am understanding the column numbers/order and the column names

#Column name + datatype mapping
print(rawdf1.dtypes) #Realizing the datatype of every columns (even we can do programattic column & type identification for dynamic programming)
  

In [0]:
#Identifying all string columns (dynamic logic)
for i in rawdf1.dtypes:
    if i[1]=='string':
        print(i[0])

#Full structural metadata
print(rawdf1.schema)#To identify the structure of the data in the StructType and StructField format

distinct() and dropDuplicates() both remove duplicate rows across all columns when no subset is provided. 
- The key difference is that dropDuplicates() allows deduplication based on specific columns, making it more flexible for data engineering use cases.   ex: rawdf1.dropDuplicates(['id'])

In [0]:
#Important passive EDA data functions we can use
#We identified few patterns on this data
#1. Deduplication of rows and given column(s)
#2. Null values ratio across all columns
#3. Distribution (Dense) of the data across all number columns
#4. Min, Max values
#5. StdDeviation - 
#6. Percentile - Distribution percentage from 0 to 100 in 4 quadrants of 25%

#Total row count (baseline)
print("actual count of the data",rawdf1.count()) 

#de duplicate the entire columns of the given  dataframe(SQL-style operation)
print("de-duplicated record (all columns) count sqlstyle",rawdf1.distinct().count())

#de duplicate the entire columns of the given  dataframe(DataFrame-specific API)
print("de-duplicated record (all columns) count DF api",rawdf1.dropDuplicates().count())

#de duplicate the entire columns of the given  dataframe(remove duplicates based on specific columns)
print("de-duplicated given cid column count",rawdf1.dropDuplicates(['id']).count())

#describe() provides basic statistics like count, mean, min, and max, 
display(rawdf1.describe())

# while summary() extends this by adding percentile-based distribution metrics such as median and quartiles, making it more suitable for deeper data quality analysis.
display(rawdf1.summary())

### **Active Data Munging** is the continuous process of structuring, validating, cleansing, scrubbing, deduplicating, and standardizing evolving data to make it analytics-ready.

- Combining Data + Schema Evolution/Merging (Structuring)
- Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)
- De Duplication and Levels of Standardization () of Data to make it in a usable format (Dataengineers/consumers)

1)**Questions related to multiple files/paths/sub path handling**
-->I have data in different filenames in a single/multiple location, i need to read all these data in a df - path=["path1/file1","path1/file2","path2/file3"] I have data in single pattern of file names in a single/multiple locations or subfolders, i need to read all these data in a df - path=["path1/","path1/","path2/"], pathGlobFilter="custsm*", recursiveFileLookup=True

2)**Questions related handling evolving data structure with data ingested in different days/periods - Ans. Schema Evolution**
Evolution is growth over the time (Filesystem level).. Eg. Source is sending data with additional columns week over week in csv format
1. Read and write in Serialized format( ORC,Parquet)
2. Read DF with mergeSchema = True

3)**Questions related handling data from different sources with different related structure in a same day - Ans. Schema Merging/Melting (Dataframe level)**    Eg. Source1 is sending custsmodified_NY with 5 columns and Source2 is sending custsmodified TX with 4 columns
1. Read file1 in DF1, read file2 in DF2
2. Create DF3 by merging DF1 and DF2 using df1.unionByName(df2,allowMissingColumns=True)

In [0]:
#Extraction (Ingestion) methodologies
#1. Single file
struct1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(struct1).csv(path="/Volumes/breadandbutter/data_ingestion_db/data_de_vl/custsmodified") 
#2. Multiple files (with different names)
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/we47catalog/we47schema/we47_volume/custsmodified","/Volumes/breadandbutter/data_ingestion_db/data_de_vl/custsmodified_NY"])
#3. Multiple files in multiple paths or sub paths
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/breadandbutter/data_ingestion_db/data_de_vl/","/Volumes/breadandbutter/data_ingestion_db/data_de_vl/"],recursiveFileLookup=True,pathGlobFilter="custsm*")

- When you go for Schema Merging/Melting and Schema Evolution?
- Schema Merging/Melting (unionByName,allowMissingColumns)- If we get multiple files
- Schema Evolution (orc/parquet with mergeSchema) - If no. of columns are keeps added by the source system
- when we know structure of the file already - schema merge/ schema not known earlier  - schema evolution

Schema Evolution:
Handling schema changes over time in the same dataset.

Schema Merging:
Combining different schemas from multiple sources into one structure.

- If multiple files with different structures arrive together → Schema Merging.
- If the same source keeps adding columns over time → Schema Evolution.

### **1. Combining Data + Schema Evolution/Merging (Structuring)**

**Schema Merging**
- Schema Merging is the process of combining data from multiple sources with different schemas at the same point in time into a single unified structure.

**Scope**
- Same-day / same batch
- DataFrame level
- _Typical Scenario_
- Source A (NY): id, name, email
- Source B (TX): id, name, phone

**_How it is handled_**
- Read separately
- Merge using unionByName
- df_all = df_ny.unionByName(df_tx, allowMissingColumns=True)


What Spark does
- Matches columns by name
- Adds missing columns as NULL
- Produces unified DataFrame

Key Point
- Schema Merging happens because sources differ, not because time changes.

In [0]:
#COMBINING OR SCHEMA MERGING or SCHEMA MELTING of Data from different sources(Important interview question also as like schema evolution...)
#4. Multiple files with different structure in multiple paths or sub paths
strt1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(strt1).csv(path=["/Volumes/breadandbutter/data_ingestion_db/data_de_vl/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_N*")

strt2="id string, firstname string, age string, profession string,city string"
rawdf2=spark.read.schema(strt2).csv(path=["/Volumes/breadandbutter/data_ingestion_db/data_de_vl/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_T*")
display(rawdf1)
display(rawdf2)

rawdf_merged=rawdf1.union(rawdf2)#Use union only if the dataframes are having same columns in the same order with same datatype....Union is position-based ,Same column order, Same number of columns, Same data types.
display(rawdf_merged)

#Expected right approach to follow #allowMissingColumns=True -> Adds missing columns with NULL
rawdf_merged=rawdf1.unionByName(rawdf2,allowMissingColumns=True) #In unionByName -> Columns matched by name # Missing columns → NULL
display(rawdf_merged) #done




In [0]:
#Here, we are merging two files because both are in CSV format. If one file is CSV and the other file is in a different format, what should we do in this scenario? it will be handled automatically
#rawdf2.write.json("/Volumes/workspace/wd36schema/ingestion_volume/staging/csvjson")
rawdf3=spark.read.json("/Volumes/breadandbutter/data_ingestion_db/data_de_vl/simple_json.txt")
rawdf_merged=rawdf_merged.unionByName(rawdf3,allowMissingColumns=True)
display(rawdf_merged)#Expected dataframe to proceed further munging on a single dataframe

### 2. Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)

%md
READ MODE's
### 1️⃣ mode='permissive' (DEFAULT)
What it does
- Reads all records
- Corrupt / malformed rows are not dropped
- They are placed into a special column called _corrupt_record

2️⃣ **mode='dropMalformed'**
 What it does
- Drops malformed rows
- No _corrupt_record column

3️⃣ **mode='failFast'**
 What it does
- Fails immediately when malformed record is found
- Stops job execution

In [0]:
#Validation by doing cleansing
from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType
#print(rawdf1.schema)
struttype1=StructType([StructField('id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('profession', StringType(), True)])

#method1 - permissive with all rows with respective nulls
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/breadandbutter/data_ingestion_db/data_de_vl/custsmodified_NY",mode='permissive')
print("after keeping nulls on the wrong data format", cleandf1.count())#all rows count
display(cleandf1)#We are making nulls where ever data format mismatch is there (cutting down mud portition from potato)
#or
#method2 - drop malformed rows
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/breadandbutter/data_ingestion_db/data_de_vl/custsmodified_NY",mode='dropMalformed')
print("after cleaning wrong data (type mismatch, column number mismatch)", len(cleandf1.collect()))
display(cleandf1)#We are removing the entire row, where ever data format mismatch is there (throwing away the entire potato)


### Validation

In [0]:
#method3 best methodology of applying active data munging
#Validation by doing cleansing (not at the time of creating Dataframe, rather we will clean and scrub subsequently)...
struttype1 = StructType([StructField('id', StringType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', StringType(), True), StructField('profession', StringType(), True)])
#method1 - permissive with all rows with respective nulls
rawdf1=spark.read.schema(struttype1).csv(path="/Volumes/breadandbutter/data_ingestion_db/data_de_vl/custsmodified_NY",mode='permissive')
print("allow all data showing the real values",rawdf1.count())#all rows count
display(rawdf1)#We are making nulls where ever data format mismatch is there (cutting down mud portition from potato)

### **Rejection Strategy**

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Schema with corrupt record column
strt11 = StructType([
    StructField("id", IntegerType(), True),
    StructField("firstname", StringType(), True),
    StructField("lastname", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("profession", StringType(), True),
    StructField("corruptdata", StringType(), True)
])

# Read data with PERMISSIVE mode
dfmethod4 = spark.read \
    .schema(strt11) \
    .option("mode", "PERMISSIVE") \
    .option("columnNameOfCorruptRecord", "corruptdata") \
    .option("header", "false") \
    .csv("/Volumes/breadandbutter/data_ingestion_db/data_de_vl/custsmodified_NY")

display(dfmethod4)

print("Entire count of data:", dfmethod4.count())

# Rejected records
df_reject = dfmethod4.filter("corruptdata IS NOT NULL")

# Write rejected data (excluding corrupt column)
df_reject.drop("corruptdata") \
    .write \
    .mode("overwrite") \
    .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv/")
print("Data to reject or update the source",len(dfmethod4.where("corruptdata is not null").collect()))
display(dfmethod4.filter("corruptdata IS NOT NULL")) #see rejected data
#print df_reject.count()

In [0]:
#Creating rejection dataset to send to our source system for future fix
from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType
struttype1=StructType([StructField('id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('profession', StringType(), True),StructField("corruptedrows",StringType())])
#method1 - permissive with all rows with respective nulls
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/breadandbutter/data_ingestion_db/data_de_vl/custsmodified_NY",mode='permissive',columnNameOfCorruptRecord="corruptedrows")
#Create a reject dataset
rejectdf1=cleandf1.where("corruptedrows is not null")
#display(rejectdf1)
rejectdf1.write.csv("/Volumes/breadandbutter/data_ingestion_db/data_de_vl/reject_data/",mode="overwrite",header=True)
retaineddf1=cleandf1.where("corruptedrows is null")
print("Overall rows in the source data is ",len(cleandf1.collect()))
print("Rejected rows in the source data is ",len(rejectdf1.collect()))
print("Clean rows in the source data is ",len(retaineddf1.collect()))

In [0]:
#Before actively Cleansing or Scrubbing - We have to create a Rejection Strategy to reduce data challenges in the future
strt11=StructType([StructField("id",IntegerType(),True),StructField("firstname",StringType(),True),StructField("lastname",StringType(),True),StructField("age",IntegerType(),True),StructField("profession",StringType(),True),StructField("corruptdata",StringType())])
dfmethod4=spark.read.schema(strt11).csv("/Volumes/breadandbutter/data_ingestion_db/data_de_vl/custsmodified_NY", mode="PERMISSIVE",header=False,columnNameOfCorruptRecord="corruptdata")
display(dfmethod4)
print("entire count of data",dfmethod4.count())
df_reject=dfmethod4.where("corruptdata is not null")
df_reject.drop("corruptdata").write.mode("overwrite").csv("/Volumes/breadandbutter/data_ingestion_db/data_de_vl/reject_data/",mode="overwrite")
#print("Data to reject or update the source",len(dfmethod3.where("corruptdata is not null").collect()))

**Cleansing**
It is a process of cleaning/removing/deleting unwanted data
### - **na.drop(how="any")** 
- Drops the entire ROW if ANY column in that row is NULL
- Only rows with NO nulls at all survive

### - 2️⃣ **na.drop(how="any", subset=["id","age"])**
- Drops row ONLY IF id OR age is NULL
- Other columns are ignored.

- Mode	              Row dropped when
- how="any"	   ----> drop if At least one column is NULL
- how="all"	   ----> drop if All selected columns are NULL


In [0]:
#We already know how to do cleansing applying the strict Structure on method1 and method2
#Important na functions we can use to do cleansing
display(rawdf1.where("age is null")) #raw data before cleansing
cleanseddf=rawdf1.na.drop(how="any")#This function will drop any column in a given row with null otherwise this function returns rows with no null columns
display(cleanseddf.where("age is null"))#after cleansing no null row data will be seen

cleanseddf=rawdf1.na.drop(how="any",subset=["id","age"]) #Drops row ONLY IF id OR age is NULL
display(cleanseddf)
cleanseddf=rawdf1.na.drop(how="all",subset=["lastname","profession"])#4000004,Gretchen,,66,
display(cleanseddf)

**Scrubbing**
 It is a process of polishing/fine tuning/scrubbing/meaningful conversion the data in a usable format
### -  **na.fill()** - Replaces NULL values in the specified column(s) with the given value
### > #na.replace() is used to replace values in a DataFrame.

In [0]:
#Replaces NULL values only in column lastname with 'user not provided' if NULL values are present Other columns remain unchanged.
cleanseddf = rawdf1.na.fill('user not provided',subset=["lastname"])#4000004,Gretchen,,66,
display(cleanseddf)

cleanseddf = rawdf1.na.fill('NA')#replaces all null values with 'NA'
display(cleanseddf)
#na.replace() is used to replace values in a DataFrame.
find_replace_values_dict1={'Pilot':'Captain','Actor':'Celeberity'}
find_replace_values_dict2={'not provided':'NA'}
#Replace values in profession column #Replaces values based on dictionary mapping
cleanseddf2=cleanseddf.na.replace(find_replace_values_dict1,subset=["profession"]) #Pilot replaced with Captain
display(cleanseddf2)
scrubbeddf3=cleanseddf2.na.replace(find_replace_values_dict2,subset=["lastname"])
display(scrubbeddf3)


###  De-duplication?
- De-duplication is the process of removing duplicate records from a dataset.
- 2.Two Types of De-duplication
### >  A. Non-Priority De-duplication
- No rule to decide which record is better.
- You just want one copy.
### > B. Priority-Based De-duplication
- Business rules decide which record to keep.
| Step                     | Technique          | Type           |
| ------------------------ | ------------------ | -------------- |
| distinct()               | Exact row match    | Row-level      |
| dropDuplicates(["id"])   | No priority        | Column-level   |
| orderBy + dropDuplicates | With business rule | Priority-based |


In [0]:
#1.Row-level (full row) de-duplication
display(scrubbeddf3.where("id in ('400001')"))  # before row level dedup
dedupdf1 = scrubbeddf3.distinct() #Removes rows that are 100% identical across all columns #Row-level/Non-priority de-duplication
display(dedupdf1.where("id in ('4000001')")) #Verify after row-level de-duplication

#2.Column-level de-duplication (Non-priority)
print("non prioritized deduplication, just remove the duplicates retaining only the first row")
display(dedupdf1.coalesce(1).where("id in ('4000003')"))#before col level dedup
#coalesce() is used to reduce the number of partitions in a Spark DataFrame or RDD.
#dropDuplicates() is used to remove duplicate rows from a Spark DataFrame
dedupdf2=dedupdf1.coalesce(1).dropDuplicates(subset=["id"])#It will remove the column level duplicates (retaining the first row in the dataframe) #Only one row per id
display(dedupdf2.where("id in ('4000003')"))

#3.Priority-based de-duplication logic
print("prioritized deduplication based on age")
display(dedupdf1.coalesce(1).where("id in ('4000003')"))#before col level dedup
dedupdf1.coalesce(1).where("id in ('4000003')").orderBy(["id","age"],ascending=[True,False]).show(3)
#manually controlling which row becomes first by: ->Sorting (orderBy) ->Then dropping duplicates
dedupdf2=dedupdf1.coalesce(1).orderBy(["id","age"],ascending=[True,False]).dropDuplicates(subset=["id"]) #It will remove the column level duplicates (retaining the first row in the dataframe)
display(dedupdf2.where("id in ('4000003')"))

### 3. Standardization and Replacement / Deletion of Data to make it in a usable format
- Making the data more standard by adding/removing/reordering columns as per the expected standard, unifying into expected format, converting the type as expected etc.,

### Standardization1 - Column Enrichment (Addition of columns)
- withColumn() is used to add a new column or replace an existing column in a DataFrame
> - df.withColumn("country", lit("IND"))
- lit() creates a literal (constant) value

In [0]:
from pyspark.sql.functions import lit,initcap,col
#withColumn("stringcolumnname to add in the df",lit('hardcoded')/initcap(col("colname")))
standarddf1=dedupdf2.withColumn("sourcesystem",lit("Retail"))#SparkSQL - DSL(FBP)
display(standarddf1.limit(20))

### Standardization2 - Column Uniformity

In [0]:
from pyspark.sql.functions import upper
#Basic Exploration/analysis of the profession column for identifying uniformity challenges
#standarddf1.createOrReplaceTempView("sqlview")
#display(spark.sql("select profession,count(*) from sqlview group by profession order by profession"))#SQL
#display(standarddf1.groupBy("profession").count())#DSL
#Standardization2 - column uniformity
#------initcap()  data scientist → Data Scientist--------------
standarddf2=standarddf1.withColumn("profession",initcap("profession"))#inicap or any other string function with columnOr name can accept either column or string type provided if the string is a column name for eg. profession/age/sourcesystem.
display(standarddf2.limit(20))
#display(standarddf2.groupBy("profession").count())#DSL

### Standardization3 - Format Standardization

In [0]:
#Did analysis to understand the format issues in our id and age columns
#standarddf2.where("id like 't%'").show() -------Finds rows where id contains letters ---> [a-zA-Z] → matches any alphabet
standarddf2.where("id rlike '[a-zA-Z]'").show()#rlike is regular expression like function that help us identify any string data in our DF column #[^0-9] → any character(a-z) that is NOT a digit  #[0-9] → any digit   # ^ → NOT
standarddf2.where("age rlike '[^0-9]'").show()#checking for any non number values in age column
#standarddf3=standarddf2.withColumn

In [0]:
from pyspark.sql.functions import regexp_replace,replace
#Let's apply scrubbing features to our id column to replace ten with 10 (or we can think of using GenAI here) ---Convert words → digits
replaceval={'one':'1','two':'2','three':'3','four':'4','five':'5','six':'6','seven':'7','eight':'8','nine':'9','ten':'10'}
standarddf3=standarddf2.na.replace(replaceval,["id"])
#standarddf3=standarddf2.withColumn("id",replace("id",lit('ten'),"10")) 
#3-0 -> 30 #regexp_replace() → pattern based
standarddf3=standarddf3.withColumn("age",regexp_replace("age",'-',""))
display(standarddf3)

### Standardization4 - Data Type Standardization
---standarddf3.withColumn("id", **col**("id").**cast**("long"))

In [0]:
standarddf3.printSchema()#Still id and age are string type, though it contains int data
#standarddf4=standarddf3.withColumn("id","id".cast("long"))#this wil not work
standarddf4=standarddf3.withColumn("id",standarddf3.id.cast("long"))
standarddf4=standarddf3.withColumn("id",standarddf3["id"].cast("long"))
standarddf4=standarddf3.withColumn("id",col("id").cast("long")) #best to use
standarddf4=standarddf4.withColumn("age",col("age").cast("short"))
standarddf4.printSchema()
display(standarddf4)

Standardization5 - Naming Standardization
- withColumnRenamed(oldName, newName) renames ONE column at a time

In [0]:
standarddf5=standarddf4.withColumnRenamed("id","custid")
standarddf5=standarddf4.withColumnsRenamed({"id":"custid","sourcesystem":"srcsystem"})
display(standarddf5)

Standardization6 - Reorder Standadization

In [0]:
standarddf6=standarddf5.select("custid", "age", "firstname","lastname","profession","srcsystem")
#display(standarddf6)
mungeddf=standarddf6
display(mungeddf.take(10))

Before starting Data Enrichment or before sharing the data to the consumer, we have to do EDA/Exploration/Validation

In [0]:
mungeddf.printSchema()
display(mungeddf.take(20))
display("total rows",len(mungeddf.collect()))
display(mungeddf.summary())

### 2. Data Enrichment - Detailing of data
- Makes your data rich and detailed
- a. Add (withColumn,select,selectExpr), Derive (withColumn,select,selectExpr), Remove(drop,select,selectExpr), Rename (withColumnRenamed,select,selectExpr), Modify/replace (withColumn,select,selectExpr) - very important spark sql functions
- b. split, merge/Concat
- c. Type Casting, reformat & Schema Migration
- Data enrichment ensures data is business-ready, analytics-ready, and schema-compliant before it reaches downstream consumers.

![](path)

![](/Workspace/Users/sachinchitte4@gmail.com/Pratice/stage2.png)

**a. Add (), Derive (), Rename (), Modify/replace (), Remove/Eliminate () - very important spark sql DF functions**

Adding of columns
Lets add datadt (date of the data orgniated from the source for eg. provided in the filename in a format of yy/dd/MM) and loaddt (date when we are loading the data into our system)

In [0]:
derived_datadt='25/30/12'
print(f"hello '{derived_datadt}'")

In [0]:
from pyspark.sql.functions import lit,current_date#already imported, not needed here
original_filename = 'custsmodified_25/30/12.csv'
derived_datadt = original_filename.split('_')[1].split('.')[0]
print(f"hello '{derived_datadt}'") #derived_datadt='25/30/12'#We are deriving this date from the filename provided by the source custsmodified_25/30/12.csv
enrichdf1=mungeddf.withColumn("datadt",lit('25/30/12')).withColumn("loaddt",current_date()) #recommended way
enrichdf1.printSchema() #lit('25/30/12') is a string literal  #current_date() returns a date type
#or
enrichdf1=mungeddf.withColumns({"datadt":lit('25/30/12'),"loaddt":current_date()})
enrichdf1.printSchema()
#or
enrichdf1=mungeddf.select("*",lit(derived_datadt).alias('datadt'),current_date().alias('loaddt'))#DSLs (FBP function)
#or
enrichdf1=mungeddf.selectExpr("*","'25/30/12' as datadt","current_date() as loaddt")#DSL(select) + SQL expression
enrichdf1=mungeddf.selectExpr("*",f"'{derived_datadt}' as datadt","current_date() as loaddt")#DSL(select) + SQL expression
enrichdf1.printSchema()
display(enrichdf1)


**Deriving of columns**
- Engineer → E
- Doctor   → D

**- substring(colName, pos, len)**

In [0]:
from pyspark.sql.functions import *
enrichdf2=enrichdf1.withColumn("professionflag",substring("profession",1,1)) #recomendded
#or
enrichdf2=enrichdf1.select("*",substring("profession",1,1).alias("professionflag"))
#or
enrichdf2=enrichdf1.selectExpr("*","substr(profession,1,1) as professionflag")
display(enrichdf2.take(20))

**Renaming of columns**
- **withColumnRenamed(oldName, newName)**

In [0]:
#Can we use withColumn to rename? not directly, its costly
enrichdf3=enrichdf2.withColumn("sourcename",col("srcsystem"))
enrichdf3=enrichdf3.drop("srcsystem").select("custid","age","firstname","lastname","profession","sourcename","datadt","loaddt","professionflag")
#or
enrichdf3=enrichdf2.select("custid","age","firstname","lastname","profession",col("srcsystem").alias("sourcename"),"datadt","loaddt","professionflag")#costly too, since we have to choose all columns in the select
#or
#enrichdf2.printSchema()
enrichdf3=enrichdf2.selectExpr("custid","age","firstname","lastname","profession","srcsystem as sourcename","datadt","loaddt","professionflag")#costly too, since we have to choose all columns in the select
#or
enrichdf3=enrichdf2.withColumnRenamed("srcsystem","sourcename")#Best function to rename the column(s)
#or
enrichdf3=enrichdf2.withColumnsRenamed({"srcsystem":"sourcename","professionflag":"profflag"})
display(enrichdf3.take(20))

Modify/replace (withColumn, select/selectExpr)
- withColumn() is used to ADD, REPLACE, or MODIFY columns.
- withColumn() is used to add or overwrite columns. If the column exists, Spark replaces it

In [0]:
enrichdf4=enrichdf3.withColumn("profession",col("sourcename"))#This will replace the profession with sourcename
#or
enrichdf4=enrichdf3.withColumn("profession",concat("profession",lit('-'),"profflag"))#This will modify/enrich the profession column with sourcename
#or using select/selectExpr
enrichdf4=enrichdf3.select("custid","age","firstname","lastname",concat("profession",lit('-'),"profflag").alias("profession"),"sourcename","datadt","loaddt","profflag")
#or use selectExpr
enrichdf4=enrichdf3.selectExpr("custid","age","firstname","lastname","concat(profession,'-',profflag) as profession","sourcename","datadt","loaddt","profflag")
display(enrichdf4.take(20))

### Remove/Eliminate (drop,select,selectExpr)
- 1️⃣ drop()  ❌ Remove columns directly (simplest)
- 2️⃣ select() ✅ Keep only required columns
- 3️⃣ selectExpr() – 🧠 SQL-style elimination + transformation

1️⃣ drop() — Column Elimination Only
- 📌 What it really does
- Removes specified columns from the logical plan
- Internally Spark applies column pruning
- No transformation, no renaming, no expressions

2️⃣ select() — Explicit Projection (Most Important)
- 📌 What it really does
- Defines exactly which columns Spark must read
- Enables predicate & projection pushdown
- Preferred by Spark optimizer (Catalyst)

 3️⃣ selectExpr() — SQL Projection Layer
- 📌 What it really does
- Parses SQL expressions into Spark logical plan
- Runs after SQL parser, before Catalyst
- Same execution as select() (performance equal)

In [0]:
#enrichdf4=enrichdf3.withColumn("profession",col("sourcename"))#Cannot be used
#or using select/selectExpr (yes, but costly)
enrichdf5=enrichdf4.select("custid","age","firstname","lastname","profession","sourcename","datadt","loaddt")
#or use selectExpr (yes, but costly)
enrichdf5=enrichdf4.selectExpr("custid","age","firstname","lastname","profession","sourcename","datadt","loaddt")
#or 
enrichdf5=enrichdf4.drop("profflag")#right function to use from dropping #Basic drop() –Remove one column
display(enrichdf5.take(20))

In [0]:
#how to write a python program to append a variable value to another variable and use it inside the selectExpr
name='irfan'
sqlexpression=f"'{name}' as owner"
print(sqlexpression)
mungeddf.selectExpr("*",sqlexpression).display()

######Conclusion/Best practices of using different column enrichment functions
1. **select** is good to use if we want to perform - 
Good for ordering/reordering of columns, only renaming column (not good), only reformatting/deriving a column (not good), **for all of these operation in a single iteration** such renaming, reordering, reformatting,deriving, dropping etc., (best to use)
2. **selectExpr** is good to use if we want to perform - Same as select by using ISO/ANSI SQL functionality (if we are not familiar in DSL FBP) **for all of these operation in a single iteration**
3. **withColumn** is good to use if we want to perform - 
**for adding/deriving/modifying/replacing in a single iteration**
Adding/Deriving column(s) in the last (Good), Modifying/replacing (Good), Renaming (not good), Dropping(not possible), reordering(not good)
4. **withColumnRenamed** is good to use if we want to perform - only for renaming column (Good)
5. **drop** is good to use if we want to perform - only dropping of columns in the given position (Good)

b. Splitting & Merging/Melting of Columns

In [0]:
#Splitting of column
splitdf=enrichdf5.withColumn("profflag",split("profession",'-'))
splitdf=splitdf.withColumn("profession",col("profflag")[0])
#splitdf=splitdf.withColumn("profflag",col("profflag")[1])
#or
splitdf=splitdf.withColumn("shortprof",upper(substring(col("profession"),1,3))).drop("profflag")
#Merging of column
mergeddf=splitdf.select(col("custid"),"age",concat_ws(" ",col("firstname"),col("lastname")).alias("fullname"),"profession","sourcename","datadt","loaddt","shortprof")#usage of select will help us avoid chaining of withColumn,drop,select
display(mergeddf.limit(10))

**B. Splitting & Merging/Melting of Columns**

In [0]:
#Splitting of column
splitdf=enrichdf5.withColumn("profflag",split("profession",'-'))
splitdf=splitdf.withColumn("profession",col("profflag")[0])
#splitdf=splitdf.withColumn("profflag",col("profflag")[1])
#or
splitdf=splitdf.withColumn("shortprof",upper(substring(col("profession"),1,3))).drop("profflag")
#Merging of column
mergeddf=splitdf.select(col("custid"),"age",concat_ws(" ",col("firstname"),col("lastname")).alias("fullname"),"profession","sourcename","datadt","loaddt","shortprof")#usage of select will help us avoid chaining of withColumn,drop,select
display(mergeddf.limit(10))

In [0]:
mergeddf.printSchema()
#unconsious and incompetant(day1)
#consious and incompetant(month3)-current state
#consious and competant(further month)-another few month state
#unconsious and competant(further month)-near end of every stage (pyspark+databricks(sql/python/dwh))

**c. Formatting & Typecasting**

1️⃣ Schema Evolution
- Schema Evolution is the ability of a system to handle changes in data structure over time as new data arrives with additional or modified columns, without breaking existing pipelines.
### Scope
- Time-based (day over day / week over week)
- Filesystem / table level
- _Typical Scenario_
- Week 1 file: id, name
- Week 2 file: id, name, email
- Week 3 file: id, name, email, phone
### How it is handled
- Use schema-aware formats (Parquet / ORC)
- Enable mergeSchema = true at read time
- _spark.read.option("mergeSchema", "true").parquet("/data/customers")_
### What Spark does
- Reads schema from all files
- Merges them into a superset
- Missing columns → NULL
- Key Point
- Schema Evolution happens because data changes over time.

In [0]:
formatteddf=mergeddf.withColumn("datadt",to_date(col("datadt"),'yy/dd/MM'))#25/30/12 -> 2025-12-30
formatteddf.printSchema()
display(formatteddf.take(10))

### 3. Data Customization - Application of Tailored Business specific Rules
- a. User Defined Functions
   -  A UDF allows you to write custom logic that Spark does not provide out-of-the-box. 
- b. Building of Frameworks & Reusable Functions (We will learn very next)


![](/Workspace/Users/sachinchitte4@gmail.com/Pratice/stage3.png)

In [0]:
#formatteddf2=formatteddf.withColumn("sourcename",upper("sourcename"))
#formatteddf2.show(2)
#Caveat - If there is no upper() function is available already in spark dsl/sql, we can either search for some functions in the online opensource platform or we have to create one (custom functions)
#from org.apache.sql.functions import upperodd

def upperodd(colname_containsvalue):
    convertedcolvalue=colname_containsvalue.upper()
    return convertedcolvalue
print(upperodd("irfan"))

In [0]:
formatteddf2=rawdf1.withColumn("firstname",upper(col("firstname")))#we can't run python function as it is
formatteddf2.explain()
#display(formatteddf2.take(10))#prefer
from pyspark.sql.functions import udf
udfupper=udf(upperodd)#promote normal python function to spark ready udf
formatteddf2=rawdf1.withColumn("firstname",udfupper(col("firstname")))#if udf is inevitable, then we create despite of performance bottleneck
formatteddf2.explain()
#display(formatteddf2.take(10))

Create Python Custom Function with complex logics

In [0]:
#Calculating age category from the given age of the customer
def pythonAgeCat(dfcol):
    if dfcol is None:
        return "Unknown"
    elif dfcol<=10:
        return "child"
    elif dfcol>10 and dfcol<=18:
        return "teenager"
    elif dfcol>18 and dfcol<=30:
        return "young"
    elif dfcol>30 and dfcol<=50:
        return "middleaged"
    else:
        return "senior"

### Import udf library, Convert to UDF, Apply in the DF

In [0]:
from pyspark.sql.functions import udf
sparkudfageCat=udf(pythonAgeCat)
customdf=formatteddf.withColumn("agecat",sparkudfageCat("age"))
display(customdf.take(10))


### 4. Data Curation/Processing (Pre Wrangling Stage) - Applying different levels of business logics, transformation, filtering, grouping, aggregation and limits applying different transformation functions
- Select, Filter
- Derive flags & Columns
- Format
- Group & Aggregate
- Limit

![](/Workspace/Users/sachinchitte4@gmail.com/Pratice/stage3.png)

- 1.**Select, Filter**
- In terms of Performance Optimzation - I ensured to do Push Down Optimization by doing select(project) & Filter(predicate) of what ever the expected data

In [0]:
#Select
#select, functions, case, literal ,from,where,group by, having, order by, limit...
#Select few columns by filtering few rows
selectdf=customdf.select("custid","age","agecat",col("profession").alias("prof"),"agecat")#DSL Select
selectdf.show(5)
selectdf=customdf.selectExpr("custid","age","agecat","profession as prof","agecat")#SQL Select
selectdf.show(5)

In [0]:
#Filter/Where - both are literally same (filter will be used by FBP developers & where will be used by SQL developers)
filterdf=selectdf.filter((col("age")>40) & (col("age")<=60))#DSL operation
filterdf.show(5)
filterdf=selectdf.where((col("age")>40) & (col("age")<=60))#DSL operation
filterdf.show(5)

filterdf=selectdf.filter("age>40 and age<=60")#SQL where operation
filterdf.show(5)
filterdf=selectdf.where("age>40 and age<=60")#SQL where operation
filterdf.show(5)
#filterdf.write.saveAsTable("filtercust")

**2. Derive flags & Columns**

In [0]:
#We have created agecat using UDF (which is supposed to use only if it is inevitable)
#But we can do the same using DSL When.otherwise or SQL CASE WHEN
#Deriving Flag
#Syntax in DSL: when(conditions,"value").when(conditions,"value2").otherwise("valuen").alis("colname")
curateddf=customdf.select("*",when(col("age").isNull(),"U").
                          when(col("age")<=10,"C").
                          when((col("age")>10) & (col("age")<=18),"T").
                          when((col("age")>18) & (col("age")<=30),"Y").
                          when((col("age")>30) & (col("age")<=50),"M").
                          otherwise("S").alias("agecatflag"))#Suggessted than using UDFs
display(curateddf.take(10))
#Deriving Column
#Syntax in SQL: case when conditions then value when conditions then value2 else valuen end as colname
curateddf=curateddf.drop("agecat").selectExpr(
    "*","""case when age is null then 'Unknown' 
                              when age<=10 then 'child' 
                              when age>10 and age<=19 then 'teenager' 
                              when age>19 and age<=30 then 'young'
                              when age>30 and age<=50 then 'middleaged'
                              else 'oldaged' end as agecat""")#Suggessted than using UDFs
display(curateddf.take(10))
#drop("agecat")-> Removes the existing agecat column (if present)
#Interview Answer of how you optimized the existing spark code developed by your ex team members?
#I analysed the existing udfs used in my project and seeked for opurtunities to convert them into SQL/dsl based programs by implementing the udf logics.

In [0]:
'''def pythonAgeCat(dfcol):
    if dfcol is None:
        return "Unknown"
    elif dfcol<=10:
        return "child"
    elif dfcol>10 and dfcol<=18:
        return "teenager"
    elif dfcol>18 and dfcol<=30:
        return "young"
    elif dfcol>30 and dfcol<=50:
        return "middleaged"
    else:
        return "senior"'''

I optimized the Spark code by identifying Python UDFs and replacing them with Spark SQL CASE WHEN and DSL when/otherwise. This removed row-level Python execution, enabled Catalyst optimization, reduced execution time, and improved scalability.

### 3.Format (Deriving Columns with different format)

In [0]:
#We can use different functions - string or number or date function for format modeling
curateddf3=curateddf.select("*",datediff("loaddt","datadt").alias("delaydays"),
                            year("datadt").alias("datayear")
                            ,month("datadt").alias("datamonth")
                            ,last_day("datadt").alias("datalastday")).withColumn("agecat",initcap("agecat"))
display(curateddf3.take(3))

### 4. Group & Aggregate
- Before performing grouping or aggr, consider the below factors from the dataset....
- identifier? cid (high in cardinality/difference) (surrogate/naturalkey)
- descriptive? name
- metric? avg(age),count(distinct cid),max(age),min(age)
- measure? age,cid
- grouping? age,prof - low in cardinality/difference

In [0]:
#from pyspark.sql.functions.aggregate import avg,count,initcap,last_day,datediff,year,month,agg
#What is the total number of customers we have?
print(curateddf3.count())
#What is the total number of customers we have in each profession?
curateddf4=curateddf3.groupBy("profession").count()
display(curateddf4.take(100))
#Multiple Aggregation with one grouping - What is the total number of customers,average age of those customers we have in each profession?
curateddf4=curateddf3.groupBy("profession").avg("age").withColumnRenamed("avg(age)","avgage")
display(curateddf4.take(100))
#To calculate multiple aggregation, we need to use a function called agg function
curateddf4=curateddf3.groupBy("profession").agg(count("custid").alias("custcount"),avg("age").alias("avgage"))
display(curateddf4.take(100))
#curateddf4 this dataframe we materialize/store in some tables/files later
#Multiple Aggregation with multiple grouping - What is the total number of customers,average age of those customers we have in each profession?
curateddf4=curateddf3.groupBy("profession","agecat").\
agg(count("custid").alias("custcount"),avg("age").alias("avgage"))
display(curateddf4.take(100))



**5. Ordering**

In [0]:
curateddf5=curateddf4.orderBy("profession","agecat")
display(curateddf5.take(100))

**6. Limit**

- Let us take an oppurtunity to understand different data limiting/restricting functions
- Limit is a dataframe TRANSFORMATION functions used to limit the number of rows returned in a spark dataframe FORMAT
- Take is a dataframe/RDD ACTION functions used to limit the number of rows returned in a python list FORMAT
- display is a standard output databricks specific function used to produce entire DF/List output in a notebook view with multiple options
- show is a standard output spark dataframe specific function used to produce default 20 rows of a DF output in a notebook/REPL/IDE view
- collect is a spark action that help us collect the DF/RDD data into the driver environment in a form of python list
| Function    | Type           | Returns        | Executes? | Safe for big data |
| ----------- | -------------- | -------------- | --------- | ----------------- |
| `limit()`   | Transformation | DataFrame      | ❌ No      | ✅ Yes             |
| `take()`    | Action         | Python list    | ✅ Yes     | ⚠ Small only      |
| `display()` | Output         | UI             | ✅ Yes     | ⚠ Limited         |
| `show()`    | Action         | Console output | ✅ Yes     | ⚠ Limited         |
| `collect()` | Action         | Python list    | ❌❌        | ❌ No              |


In [0]:
#anything can be used under display()
print("limit output")
curateddf5.limit(20).show(10)
print("take output")
curateddf5.take(10)
#When to use what
#I have to filter some data in a limited dataset of 100 rows
curateddf5.limit(100).filter("profession='Accountant'").show()
#Display
display(curateddf5.limit(100).filter("profession='Accountant'"))

**5. Data Wrangling - More of Analytics + Transformation**
- 1.Joins - Relation/Connection established between one or more datasets/df/tabl to produce the broader/extended view of the data horizontally. 5 Categories of Joins: inner, outer(left right, full), self, cross, special optimized (semi, anti)
- 2.Lookup
- 3.Lookup & Enrichment
- 4.Schema Modeling (Denormalization)
- 5.Windowing
- 6.Analytical
- 7.Set operations
- 8.grouping & aggregations

![](/Workspace/Users/sachinchitte4@gmail.com/Pratice/stage5.png)

**1. Joins**
Joins are Relation/connection of one or more tables to perform widened (horizontal) data analytics
- Frequently used simple joins (inner, left)
- _df_left.join(df_right,  on="custid",  how="inner")_
- InFrequent simple joins (self, right, full, cartesian)
- Advanced joins (Semi and Anti)
- Syntax - dfleft.join(dfright,how='typeofjoin',on='custid'=='cid')
- Optimized joins (Broadcast join, SMB Join, Shuffle hash join, Map/Reduceside join, Skewed join etc.)-AQE (Adaptive Query Execution)

In [0]:
#Ques1: In what order i can apply the functions in dataframe curation/transformation
#Although Spark’s Catalyst optimizer may reorder operations, I apply selective filters before expensive operations like na.drop() to reduce data volume early and improve performance
rawdf1=rawdf1.na.drop().where("id<>'ten' and id<>'trailer_data:end of file'")
rawdf1.explain()
rawdf1=rawdf1.where("id<>'ten' and id<>'trailer_data:end of file'").na.drop()
rawdf1.explain()
#Ques2:If I wanted to join more than 1 tables
rawdf2=rawdf1.withColumn("multiconcat",concat(concat(lit('a'),lit('b'))))
rawdf3=rawdf2
joineddf1=rawdf1.join(rawdf2,how='inner')
rawdf4=rawdf2
joineddf1.join(rawdf3,how='left').join(rawdf4,how='right')

Correct order for most pipelines
- where() / filter() → row reduction
- select() → column pruning
- na.drop() / fill() → cleanup
- withColumn() → derivations
- groupBy() / join() → heavy ops

In [0]:
from pyspark.sql.functions import lit,col
#How to write join syntax in Spark and learn the semantics of join in spark
from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType

rawdf1=spark.read.csv("/Volumes/workspace/default/usage_metrics/custsmodified",header=False,inferSchema=False).toDF("id","firstname","lastname","age","profession")
rawdf1=rawdf1.na.drop().where("id<>'ten' and id<>'trailer_data:end of file'")
rawdf1=rawdf1.where("id<>'ten' and id<>'trailer_data:end of file'").na.drop()
leftdf=rawdf1.select("id","age","firstname","lastname").where("id in (4000100,4000101)")
rightdf=rawdf1.select("id","profession").where("id in (4000100,4000102,4000103)")
leftdf.show(20,False)
rightdf.show(20,False)
#Let's understand all types of joins syntax & semantics quickly
#dfleft.join(dfright,how='typeofjoin',on='custid'=='cid')

#5 Categories of Joins : 1. inner(important), 2. outer(left(important), right, full), 3. special optimized(important) (semi, anti), 4. self, 5. cross
#1. inner, 2. outer(left, right, full)
#1. inner
#Usecase - Ensure which ever the customers (from both df) matches - Functionality(application)
print("inner")
innerjoindf=leftdf.join(rightdf,how='inner',on='id')
innerjoindf.show(20)#only 4000100 from both df is displayed

#2. outer(left, right, full)
#2. left
#Usecase - Ensure all of the master customers with/without profession provided - Functionality(application)
print("left")
leftjoindf=leftdf.join(rightdf,how='left',on='id')#syntax(how to do)
leftjoindf.show(20)#semantics(what is the output) - only 4000100,4000101 from both df is displayed with non applicable nulls in the right

#2. right
#Usecase - Ensure all of the customers with professions has to be displayed
print("left")
rightjoindf=leftdf.join(rightdf,how='right',on='id')#syntax(how to do)
rightjoindf.show(20)#semantics(what is the output) - only 4000100,4000102,4000103 from both df is displayed with non applicable nulls in the left

#2. full
#Usecase - Ensure all of the customers with/without master customers or professions has to be displayed
print("full")
fulljoindf=leftdf.join(rightdf,how='full',on='id')#syntax(how to do)
fulljoindf.show(20)#semantics(what is the output) - displays all 4000100,4000101,4000102,4000103 from both df is displayed with non applicable nulls from both left and right

#3. special optimized (semi, anti)
#Why semi/anti is an optimized join?
#Interview answer - I found low hanging fruits oppurtunities in my project.. converting inner/left joins to semi/anti, because the joins are good in performance (it uses EXISTS / NOT EXISTS condition rather than in condition)
#semi
#Usecase - Ensure which ever the customers (from both df) matches - Functionality(application)
print("left semi/semi")
semijoindf=leftdf.join(rightdf,how='leftsemi',on='id')
semijoindf.show(20)#only 4000100 (matches between both df) from LEFT df is displayed

#anti
#Usecase - Ensure which ever the customers (from both df) matches - Functionality(application)
print("left anti/anti")
antijoindf=leftdf.join(rightdf,how='leftanti',on='id')
antijoindf.show(20)#only 4000101 (un matched between both df) from LEFT df is displayed

#4. Self join
#Usecase - Hierarchical Retrival or join - Data joined by itself to produce the relational output of the self dataset
print("self")
custaffliatedf=leftdf.withColumn("refcustid",lit('4000100'))
custaffliatedf.show(3)
selfjoindf=custaffliatedf.alias("l").join(custaffliatedf.alias("r"),how='inner',on=(col('l.id')==col("r.refcustid")))

selfjoindf.select('l.id','r.*').show(20)#only 4000101 (un matched between both df) from LEFT df is displayed

#5. cross join
#What join it will perform by default? inner join (if we use on condition), cartesian/cross join (if no on condition used)
#Minimum syntax to use?
#cross
print('cartesian/cross')
joindf=leftdf.join(rightdf)#without on it is cross/cartesian product (very costly and avoidable join)
joindf.show(20)#6 rows returned

print('inner by default')
joindf=leftdf.join(rightdf,on='id')#default inner join
joindf.show(20)#4000100 returned

print('expecting right join, but because of lack of on, cross join happened')
joindf=leftdf.join(rightdf,how='right')#without on it is cross/cartesian product (very costly and avoidable join)
joindf.show(20)#6 rows returned



I identified performance improvement opportunities by converting inner and left joins into semi and anti joins wherever the requirement was only to check existence. Semi and anti joins avoid dataset widening, reduce shuffle and memory usage, and internally use EXISTS/NOT EXISTS semantics, making them much more efficient.

### Applications of Joins
Requirement: I need to do analytics/reporting of top 3 customers who did highest amount of transactions in our business last year, so i can send them some offers.

In [0]:
from pyspark.sql.types import *
#We already have customer master (Qualifying/dimension) data in a curated state
curateddf3.show(5)#This contains only customer curated data (munged, enriched, customized, curated)
#We need to bring transaction detailed (Quantifying/Fact) data upto curated state (we need to follow all/any/none of the steps that we followed so far munged, enriched, customized, curated, wrangle etc., depends on the EDA result)
#strt1="txnid long,txndt string,custid int,amt double,category string,product string,city string,state string,spendby string"
strt1=StructType([StructField('txnid', IntegerType(), True), StructField('txndt', StringType(), True), StructField('custid', IntegerType(), True), StructField('amt', DoubleType(), True), StructField('category', StringType(), True), StructField('product', StringType(), True), StructField('city', StringType(), True), StructField('state', StringType(), True), StructField('spendby', StringType(), True)])
#txnsrawdf=spark.read.csv("/Volumes/workspace/default/volumewd36/txns_2025.txt",inferSchema=True,header=False)
#txnsrawdf.schema
txnsrawdf=spark.read.schema(strt1).csv("/Volumes/breadandbutter/data_ingestion_db/data_de_vl/txns_2025.txt",header=False)
txnsrawdf.printSchema()
txnsrawdf.show(5)
txnsrawdf.summary().show(100)
#As per the above output, we can do some transformations of munging, enrichment etc.,
txnsmungeddf=txnsrawdf.na.drop().dropDuplicates(["txnid"])
txnsenrichdf=txnsmungeddf.withColumn("surrogatekey",monotonically_increasing_id())

txnscurateddf=txnsenrichdf.withColumn("txndt",to_date(col("txndt"),"MM-dd-yyyy"))
#If value = "2024/12/25"
#Format = "MM-dd-yyyy"
txnscurateddf.printSchema()
txnscurateddf.show(5)#Completed munging, enrichment and curation

#Now lets achieve the solution for the business requirement:
#Requirement: I need to do analytics/reporting of top 3 customers who did highest amount of transactions in our business last month, so i can send them some offers.
#spark.sql("select month(add_months(current_date(), -1))").show()
txnsdecdf=txnscurateddf.where("month(txndt)=month(add_months(current_date(), -1))")
print("dec transactions count ",txnsdecdf.count())

custdimdf=curateddf3
joineddf=custdimdf.join(txnsdecdf,how="inner",on="custid")
joineddf.show(10)
#We have to generate a reporting table with only custid,name,profession,amount,spendby
reportdf=joineddf.select("custid","fullname","amt","spendby").orderBy(["amt"],ascending=[False]).limit(3)
reportdf.show()#Top 3 transacting customer info (in the overall data)
reportdf=joineddf.select("custid","fullname","amt","spendby").orderBy(["amt"],ascending=[True]).limit(3)
reportdf.show()#Least 3 transacting customer info (in the overall data)


**1. Lookup (only exists/non exists check) - semi or anti**
Lookup is the process of looking up for some data attributes using the key to identify the presence of the values (not the actual values are returned) Eg. whether this particular customer made a transaction or not

In [0]:
#Lookup: Show me only the customer information, who did transactions last month (i dont need what transaction)?
#What type of best join i have to use? semi join is best and faster
custonlylastmonthtransacteddf=custdimdf.join(txnsdecdf,how='semi',on='custid')
custonlylastmonthtransacteddf.show(2)
print(custonlylastmonthtransacteddf.count())
#I store this data in a table, my business analysts will apply filter a particulatr customer

#It is possible to produce the same result by using inner or left join also, but not preferred just for lookup
custonlylastmonthtransacteddf=custdimdf.alias("cust").join(txnsdecdf,how='inner',on='custid')
resultdf=custonlylastmonthtransacteddf.select(custdimdf["*"]).distinct()
print(resultdf.count())

custonlylastmonthtransacteddf=custdimdf.alias("cust").join(txnsdecdf,how='left',on='custid')
resultdf=custonlylastmonthtransacteddf.select(custdimdf["*"]).where("txnid is not null").distinct()
print(resultdf.count())

#Lookup: Show me only the customer information, who did not do transactions last month?
custonlylastmonthnontransacteddf=custdimdf.join(txnsdecdf,how='anti',on='custid')
custonlylastmonthnontransacteddf.show(2)
print(custonlylastmonthnontransacteddf.count())

custonlylastmonthtransacteddf=custdimdf.alias("cust").join(txnsdecdf,how='left',on='custid')
resultdf=custonlylastmonthtransacteddf.select(custdimdf["*"]).where("txnid is null").distinct()
print(resultdf.count())

2. Lookup & Enrichment
Lookup and enrichment using joins - left_join (best), inner, right/full(least bother) Lookup and enrichment is the process of looking up for for some data attributes using the key and enrich the values

In [0]:
#Lookup & Enrichment: Show me both the customer information and transactions information of last month (i need what are the transactions made by the given customer)?
#What type of best join i have to use? inner (only those who did transactions) or left join (all customers)
#left
custonlylastmonthtransacteddf=custdimdf.join(txnsdecdf,how='left',on='custid')
custonlylastmonthtransacteddf.show(2)
print(custonlylastmonthtransacteddf.count())
#inner
custonlylastmonthtransacteddf=custdimdf.join(txnsdecdf,how='inner',on='custid')
custonlylastmonthtransacteddf.show(2)
print(custonlylastmonthtransacteddf.count())

#We can't achieve the same result by using semi join, because it will not return the transaction info

3.**Schema Modeling (Denormalization-joined result of tables)-DWH/DataMart**
- Use inner(mostly),left/right/full(we can use depends on the business) We can build Datawarehouse components (dimension, fact tables) appling joins on the tables to achieve different types of schemas

- Star Schema txns_fact cust_mast_dim(dim)
- id,amt cid,name,age,city_lived 1,100 11,irfan,44,Bangalore|Chennai
- Snowflake Schema - txns_fact cust_mast_dim(dim) cust_det_dim(subdim) id,amt cid,name,age cid,city_lived 1,100 11,irfan,44 11,Bangalore 11,Chennai

In [0]:
#Implementing Star schema model
#In our current dataset, we have what schema model can be defined? Only star schema is possible, because we have only one fact table and multiple dimension tables (no sub dimension).
#What is the best way to join all the tables? We can use inner join, because we have only one fact table and multiple dimension tables.
#inner
denormalized_fat_wide_df=custdimdf.join(txnsdecdf,how='inner',on='custid')
denormalized_fat_wide_df.show(2)
#I store this denormalized dataframe data into final fact table (in a single table) to simplify my customer queries without applying joins.
print(denormalized_fat_wide_df.count())

#or We can use left join also to ensure all customer info is captured.
#left
denormalized_fat_wide_df=custdimdf.join(txnsdecdf,how='left',on='custid')
denormalized_fat_wide_df.show(2)
print(denormalized_fat_wide_df.count())

#or We can use full join also to ensure all customer (and+or) transaction info is captured.
#left
denormalized_fat_wide_df=custdimdf.join(txnsdecdf,how='full',on='custid')
denormalized_fat_wide_df.show(2)
print(denormalized_fat_wide_df.count())

4.Windowing Functionalities
- Is the concept of grouping/bucketize/dividing/partitioning and performing some analytical operation on the literally partitioned data Benifits of Windowing Functionality
- 
- Creating Surrogate/primary key/seq number
- Performing Top N Analysis
- Duplicate Handling

In [0]:
from pyspark.sql.window import Window
denormalized_fat_wide_df=custdimdf.join(txnsdecdf,how='inner',on='custid')
print(denormalized_fat_wide_df.count())
orderjoineddf=denormalized_fat_wide_df.where("custid in (4000022,4000816)").select("custid","age","profession","txndt","amt","category","product","city","state","spendby")
#display(orderjoineddf)

#We are using row_number() window function (very important)
#Synax for windowing function
#from pyspark.sql.window import Window
#select(row_number().over(Window.partitionBy("custid").orderBy("txndt"))).alias("seqnum"))

#1.Creating Surrogate/primary key/seq number
#Let us perform a non windowing operation that means we are not using partitioning
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.orderBy("custid","txndt")))#overall sorting
display(sk_orderjoinedf)

#2. Performing Top N Analysis
#Let us perform windowing operation
#Interview Questions pattern:
print("a. Tell me the top 1 transaction made by the customer")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid").orderBy(desc("amt")))).where("seqnum=1")
display(sk_orderjoinedf)
print("b. Tell me the least 1 transaction made by the customer")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid").orderBy("amt"))).where("seqnum=1")
display(sk_orderjoinedf)
print("c. Tell me the top 2 transaction made by the customer")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid").orderBy(desc("amt")))).where("seqnum<=2")
display(sk_orderjoinedf)
print("d. Tell me the just the 2nd highest transaction made by the customer")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid").orderBy(desc("amt")))).where("seqnum=2")
display(sk_orderjoinedf)
print("e. Tell me the just the 2nd highest transaction made by the customer within the given state")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid","state").orderBy(desc("amt")))).where("seqnum=2")
display(sk_orderjoinedf)

print("f. Show me the very first time the business made by the given customers?")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid").orderBy("txndt"))).where("seqnum=1")
display(sk_orderjoinedf)
print("g. Show me the very first two transaction made by the given customers?")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid").orderBy("txndt")))\
.where("seqnum<=2")
display(sk_orderjoinedf)

#3. Duplicate Handling
#I want to remove duplicates based on custid with same spendby (for a given customer i need only one credit and one cash transaction info)
print("I want to get the duplicate custid with the same spendby removed out from our data?")
#display(sk_orderjoinedf.coalesce(1).dropDuplicates(["custid","spendby"]))
#Using windowing function we can do controlled way of dropping duplicates, for eg. i want to drop the repeating data of same customer and spendby by retaining only the latest transactions
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid","spendby").orderBy(desc("txndt"))))
display(sk_orderjoinedf)
display(sk_orderjoinedf.where("seqnum=1"))

#but if amt is same for 2 transaction of diff dates? - rank

In [0]:
#Second important window functions are rank() - used for applying same rank for same values and will have gaps (don't maintain continuity) 
# and dense_rank() - used for applying same rank for same values and will not have gaps (maintain denser/close continuity) 
print("a. Tell me the just the 2nd highest transaction made by the customer")
sk_orderjoinedf=orderjoineddf.withColumn("rnk",rank().over(Window.partitionBy("custid").orderBy(desc("spendby")))).withColumn("densernk",dense_rank().over(Window.partitionBy("custid").orderBy(desc("spendby"))))
display(sk_orderjoinedf)

**5.Analytical Functionalities**
Performing analytics/summarization/categorization of data by applying/not by applying windowing functionality.

In [0]:
#Comparitive analytics - lead & lag
sk_orderjoinedf=orderjoineddf.withColumn("nexttransamt",lead("amt",1,0).over(Window.partitionBy("custid").orderBy(asc("txndt")))).withColumn("priortransamt",lag("amt",1,0).over(Window.partitionBy("custid").orderBy(asc("txndt"))))
display(sk_orderjoinedf)

![](D:\Data_Engineer_docs\data_docs\stage3.png)